In [3]:
import random as r

from engine import Value
from modules import *
from ops import *
from gpt import *

In [5]:
batch_size = 2
vocab_len = 10
model_dim = 8
max_seq_len = 5
seq_len = 3
num_heads = 2
head_dim = 4
mlp_mult = 4

In [7]:
class ResidualLayer(Module):
    def __init__(self, model_dim, num_heads, head_dim, max_seq_len, mlp_mult):
        self.mhsa = MultiHeadSelfAttention(model_dim, num_heads, head_dim, max_seq_len)
        self.mlp = MultiLayerPerceptron(model_dim, mlp_mult * model_dim, model_dim)

    def __call__(self, x):
        x_normed = vector_wise_apply(layer_norm, x)
        x_mhsa = self.mhsa(x_normed)
        x_2 = entry_wise_add(x, x_mhsa)
        x_2_normed = vector_wise_apply(layer_norm, x_2)
        x_mlp = vector_wise_apply(self.mlp, x_2_normed)
        return entry_wise_add(x_2, x_mlp)

In [14]:
x = [[[Value(r.uniform(-1,1)) for _ in range(model_dim)]
      for _ in range(seq_len)]
     for _ in range(batch_size)]
print(get_shape(x))
layer = ResidualLayer(model_dim, num_heads, head_dim, max_seq_len, mlp_mult)
y = layer(x)
print(get_shape(y))

[2, 3, 8]
[2, 3, 8]
